In [12]:
import csv
import numpy as np
wine_path = "data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy= np.loadtxt(wine_path, dtype=np.float32, delimiter=";",skiprows=1)
wineq_numpy.shape

(4898, 12)

In [11]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [21]:
import torch
wineq= torch.from_numpy(wineq_numpy)
wineq.shape
wineq.type()

'torch.FloatTensor'

In [71]:
data= wineq[:,:-1]
print(data.shape)

target = wineq[:, -1].long()
target

torch.Size([4898, 11])


tensor([6, 6, 6,  ..., 6, 7, 6])

In [83]:
## target_onehot = torch.zeros(target.shape[0], 10)
print(target.unsqueeze(1).shape)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)
#r[:,0:9]
#print(target_onehot.shape)

torch.Size([4898, 1])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [84]:
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)

bad_indexes = torch.le(target, 3)
bad_indexes,bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()
bad_data = data[bad_indexes]
bad_data.shape

(tensor([False, False, False,  ..., False, False, False]),
 torch.Size([4898]),
 torch.bool,
 tensor(20))

In [88]:
bad_data = data[torch.le(target, 3)]
mid_data = data[torch.gt(target, 3) & torch.lt(target, 7)]
good_data = data[torch.ge(target, 7)]


bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)
bad_mean.shape

torch.Size([11])

In [115]:
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
 print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))
 #print(args)

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [117]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
print(total_sulfur_data.shape)
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()


torch.Size([4898])


(torch.Size([4898]), torch.bool, tensor(2727))

In [119]:
actual_indexes = torch.gt(target, 5)
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()



(torch.Size([4898]), torch.bool, tensor(3258))

In [121]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()

n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
#n_matches, n_matches / n_predicted, n_matches / n_actual

tensor(2018)
2018


## Time series data

In [147]:
bikes_numpy = np.loadtxt("data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
 dtype=np.float32,
 delimiter=",",
 skiprows=1,
 converters={1: lambda u: float(u[8:10])})
bikes = torch.from_numpy(bikes_numpy)

In [152]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()
daily_bikes.shape

daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

first_day = bikes[:24].long()
print(first_day.shape)
weather_onehot = torch.zeros(first_day.shape[0], 4)
print(weather_onehot)
first_day[:,9]

torch.Size([24, 17])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [168]:
weather_onehot.scatter_(dim=1,index=first_day[:,9].unsqueeze(1).long() - 1,value=1.0)
torch.cat((bikes[:24], weather_onehot), 1)[:1]


torch.Size([1, 21])

## NLP

In [172]:
with open('data/p1ch4/jane-austin/1342-0.txt', encoding='utf8') as f:
 text = f.read()
lines = text.split('\n')
line = lines[200]
len(line)
letter_t = torch.zeros(len(line), 128) 

70

In [173]:
def clean_words(input_str):
 punctuation = '.,;:"!?”“_-'
 word_list = input_str.lower().replace('\n',' ').split()
 word_list = [word.strip(punctuation) for word in word_list]
 return word_list
words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [187]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}
word2index_dict,len(word2index_dict), word2index_dict['impossible']

({'': 0,
  '#1342]': 1,
  '$5,000)': 2,
  "'_she": 3,
  "'after": 4,
  "'ah": 5,
  "'as-is'": 6,
  "'bingley": 7,
  "'had": 8,
  "'having": 9,
  "'i": 10,
  "'keep": 11,
  "'lady": 12,
  "'lately": 13,
  "'lydia": 14,
  "'mr": 15,
  "'my": 16,
  "'oh": 17,
  "'s": 18,
  "'this": 19,
  "'tis": 20,
  "'violently": 21,
  "'yes,'": 22,
  "'you": 23,
  '($1': 24,
  '(801)': 25,
  '(a)': 26,
  '(an': 27,
  '(and': 28,
  '(any': 29,
  '(available': 30,
  '(b)': 31,
  '(by': 32,
  '(c)': 33,
  '(comparatively': 34,
  '(does': 35,
  '(for': 36,
  '(glancing': 37,
  '(if': 38,
  '(lady': 39,
  '(like': 40,
  '(most': 41,
  '(my': 42,
  '(or': 43,
  '(trademark/copyright)': 44,
  '(unasked': 45,
  '(what': 46,
  '(who': 47,
  '(www.gutenberg.org)': 48,
  '(“the': 49,
  '*': 50,
  '***': 51,
  '*****': 52,
  '1': 53,
  '1.a': 54,
  '1.b': 55,
  '1.c': 56,
  '1.d': 57,
  '1.e': 58,
  '1.e.1': 59,
  '1.e.2': 60,
  '1.e.3': 61,
  '1.e.4': 62,
  '1.e.5': 63,
  '1.e.6': 64,
  '1.e.7': 65,
  '1.e.8': 66

In [188]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
 word_index = word2index_dict[word]
 word_t[i][word_index] = 1
 print('{:2} {:4} {}'.format(i, word_index, word))
print(word_t)
print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([11, 7261])


## Images

In [191]:
import imageio
img_arr = imageio.imread('data/p1ch4/image-dog/bobby.jpg')
img_arr.shape


(720, 1280, 3)

In [2]:

batch_size = 100
batch = torch.zeros(100, 3, 256, 256, dtype=torch.uint8)

import os

data_dir = 'data/p1ch4/image-cats/'
print(os.listdir(data_dir))
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']
print(filenames)
for i, filename in enumerate(filenames):
 img_arr = imageio.imread('data/p1ch4/image-cats/'+ filename)
 print(img_arr)


NameError: name 'torch' is not defined

## Volumetric data

In [207]:
dir_path = "data/p1ch4/volumetric-dicom/2-LUNG3B70f-04083"
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape


Reading DICOM (examining files): 1/99 files (1.0%27/99 files (27.3%63/99 files (63.6%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 14/99  (14.135/99  (35.459/99  (59.679/99  (79.898/99  (99.099/99  (100.0%)


(99, 512, 512)

In [211]:
vol = torch.from_numpy(vol_arr).float()
print(vol)
vol = torch.transpose(vol, 0, 2)
print('---')
print(vol)
vol = torch.unsqueeze(vol, 0)
vol.shape

tensor([[[ -985.,  -990.,  -999.,  ..., -1017., -1008.,  -971.],
         [-1016.,  -984.,  -963.,  ..., -1000., -1009.,  -999.],
         [-1024., -1008.,  -996.,  ...,  -979., -1021.,  -987.],
         ...,
         [ -920.,  -942.,  -944.,  ...,  -893.,  -917.,  -955.],
         [ -871.,  -879.,  -905.,  ...,  -895.,  -869.,  -867.],
         [ -876.,  -855.,  -873.,  ...,  -933.,  -982.,  -936.]],

        [[ -982.,  -989., -1020.,  ...,  -988.,  -977.,  -980.],
         [-1015.,  -969.,  -980.,  ...,  -981.,  -974., -1016.],
         [-1020.,  -978.,  -972.,  ...,  -988.,  -993., -1015.],
         ...,
         [ -880.,  -854.,  -924.,  ...,  -909.,  -829.,  -829.],
         [ -907.,  -868.,  -928.,  ...,  -909.,  -901.,  -894.],
         [ -885.,  -850.,  -899.,  ...,  -977.,  -964.,  -991.]],

        [[-1022., -1011.,  -983.,  ..., -1013.,  -980.,  -967.],
         [-1022., -1005., -1012.,  ...,  -948.,  -966., -1008.],
         [ -991.,  -965.,  -993.,  ...,  -964., -1011.,  -

torch.Size([1, 512, 512, 99])